# Metaphor Identification via RAG with OpenAI Model

This notebook details the process of metaphor identification via RAG using the OpenAI Model.

The following packages are needed to run this notebook:

In [ ]:
!pip install pandas langchain openai

Import packages.

In [ ]:
import pandas as pd
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_ollama import OllamaEmbeddings
from openai import OpenAI

Please input a test text here.

In [ ]:
key_text=""

Or, alternatively, you may choose a test text from our metaphor corpus.

(Here, we choose the first text as an example.)

In [ ]:
ds_fp="data/metaphor_dataset.csv"
ds_df=pd.read_csv(ds_fp)
ds_df=ds_df[["textid","plain"]]
ds_df.rename(columns={"plain":"context"},inplace=True)

key_text=ds_df.loc[0,"context"]

Load the metaphor protocol in plain text. This is used as the knowledge base for context retrival in the following process.

In [ ]:
total_context_fp="data/rag_context.txt"
with open(total_context_fp,"r",-1) as f:
    total_context=f.read()

Transform the text into Documents.

In [ ]:
docs=[[Document(page_content=text)] for text in [total_context]]
docs_list=[item for sublist in docs for item in sublist]

Split documents into chunks, on which the retrival is based upon. 

In [ ]:
text_splitter=RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0)
doc_splits=text_splitter.split_documents(docs_list)

Vectorizing the split documents.

In [ ]:
vectorstore = SKLearnVectorStore.from_documents(
    documents=doc_splits,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
)

Construct a retriever, and use it to retrieve the context based on the inputted test text.

In [ ]:
retriever=vectorstore.as_retriever(k=1)
retrieved_documents=retriever.invoke(key_text)
context=retrieved_documents[0].page_content

Construct the chat based on the retrieved context and the test text.

In [ ]:
p_strat=[
    {"role":"system",
     "content":"You are a helpful AI assistant. Use the following pieces of context to answer the question at the end. If you don't know the answer, just say you don't know. DO NOT try to make up an answer. If the question is not related to the context, politely respond that you are tuned to only answer questions that are related to the context.\n"},
    {'role': 'system',
    'content': 'You are a linguistic expert trained in metaphor identification. When the user provides a text, follow this protocol:\n• Identify all metaphorical expressions.\n• Wrap each one in <Metaphor> and </Metaphor> tags.\n• Reproduce the rest of the text exactly as written.\n• Do not include any explanation, commentary, or extra content in this message.'},
    {'role': 'user',
    'content': 'Can you please identify and tag the metaphors in the following text?\n'},
]
ct=p_strat.copy()
p_strat[0]["content"]=p_strat[0]["content"]+context
p_strat[-1]["content"]=p_strat[-1]["content"]+key_text

You will also need to apply for an OpenAI api key to use models from OpenAI. 

After successful application, you'll need to put your API key here:

In [ ]:
my_api_key="INSERT YOUR KEY HERE"

With API key, you can connect to the openai server via the following script:

In [ ]:
client=OpenAI(api_key=my_api_key)

Next, as the last step required before run, you need to specify a model.

The models we used in our paper are:

gpt-4.1-mini-2025-04-14

gpt-4.1-nano-2025-04-14

gpt-4.1-2025-04-14

o3-2025-04-16

o4-mini-2025-04-16

o3-mini-2025-01-31

In [ ]:
modelid="gpt-4.1-mini-2025-04-14"

Send chat to model for inferring, and retrieve result.

In [ ]:
cr=client.chat.completions.create(model=modelid,messages=ct,n=1)# choose n>1 when you need to batch testing.

rs=cr.choices[0].message.content

View the result.

In [ ]:
print(rs)